# Cluster Validation

## Import libraries for analysis

In [1]:
# Built-in libraries
import time
from itertools import product
from math import log

# NumPy, SciPy and Pandas
import numpy as np
import pandas as pd

# Scikit-Learn
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.metrics.pairwise import pairwise_distances

# Matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
combined_profiles = pd.read_csv('final_profiles.csv')

/Users/yangxiya/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (1,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Separate residential and non-residential buildings
is_residential = combined_profiles.Industry == 'Residential'
residential_profiles = combined_profiles.loc[is_residential, :]
non_residential_profiles = combined_profiles.loc[~is_residential, :]

# reset index
residential_profiles.reset_index(inplace = True, drop = True)
non_residential_profiles.reset_index(inplace = True, drop = True)

## Cluster validation functions

In [4]:
def divide(data, labels):
    clusters = set(labels)
    clusters_data = []
    for cluster in clusters:
        clusters_data.append(data[labels == cluster, :])
    return clusters_data

def get_centroids(clusters):
    centroids = []
    for cluster_data in clusters:
        centroids.append(cluster_data.mean(axis=0))
    return centroids

In [7]:
def cohesion(data, labels):
    clusters = sorted(set(labels))
    sse = 0
    for cluster in clusters:
        cluster_data = data[labels == cluster, :]
        centroid = cluster_data.mean(axis = 0)
        sse += ((cluster_data - centroid)**2).sum()
    return sse

def separation(data, labels, cohesion_score):
    # calculate separation as SST - SSE
    return cohesion(data, np.zeros(data.shape[0])) - cohesion_score

def SST(data):
    c = get_centroids([data])
    return ((data - c) ** 2).sum()

def SSE(clusters, centroids):
    result = 0
    for cluster, centroid in zip(clusters, centroids):
        result += ((cluster - centroid) ** 2).sum()
    return result

# Clear the store before running each time
within_cluster_dist_sum_store = {}
def within_cluster_dist_sum(cluster, centroid, cluster_id):
    if cluster_id in within_cluster_dist_sum_store:
        return within_cluster_dist_sum_store[cluster_id]
    else:
        result = (((cluster - centroid) ** 2).sum(axis=1)**.5).sum()
        within_cluster_dist_sum_store[cluster_id] = result
    return result

def RMSSTD(data, clusters, centroids):
    df = data.shape[0] - len(clusters)
    attribute_num = data.shape[1]
    return (SSE(clusters, centroids) / (attribute_num * df)) ** .5

# equal to separation / (cohesion + separation)
def RS(data, clusters, centroids):
    sst = SST(data)
    sse = SSE(clusters, centroids)
    return (sst - sse) / sst

def DB_find_max_j(clusters, centroids, i):
    max_val = 0
    max_j = 0
    for j in range(len(clusters)):
        if j == i:
            continue
        cluster_i_stat = within_cluster_dist_sum(clusters[i], centroids[i], i) / clusters[i].shape[0]
        cluster_j_stat = within_cluster_dist_sum(clusters[j], centroids[j], j) / clusters[j].shape[0]
        val = (cluster_i_stat + cluster_j_stat) / (((centroids[i] - centroids[j]) ** 2).sum() ** .5)
        if val > max_val:
            max_val = val
            max_j = j
    return max_val

def DB(data, clusters, centroids):
    result = 0
    for i in range(len(clusters)):
        result += DB_find_max_j(clusters, centroids, i)
    return result / len(clusters)

def XB(data, clusters, centroids):
    sse = SSE(clusters, centroids)
    min_dist = ((centroids[0] - centroids[1]) ** 2).sum()
    for centroid_i, centroid_j in list(product(centroids, centroids)):
        if (centroid_i - centroid_j).sum() == 0:
            continue
        dist = ((centroid_i - centroid_j) ** 2).sum()
        if dist < min_dist:
            min_dist = dist
    return sse / (data.shape[0] * min_dist)

def Dunn(data, clusters, centroids):
    pairwise_distances()

In [8]:
from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabaz_score

def get_validation_scores(data, labels):
    within_cluster_dist_sum_store.clear()
    
    clusters = divide(data, labels)
    centroids = get_centroids(clusters)
    
    scores = {}

    scores['cohesion'] = cohesion(data, labels)
    scores['separation'] = separation(data, labels, scores['cohesion'])
    scores['calinski_harabaz_score'] = calinski_harabaz_score(data, labels)
    scores['RMSSTD'] = RMSSTD(data, clusters, centroids)
    scores['RS'] = RS(data, clusters, centroids)
    scores['DB'] = DB(data, clusters, centroids)
    scores['XB'] = XB(data, clusters, centroids)
    
    return scores

### Test Case
- To show that the computation is correct
- By checking against manually calculated values

In [9]:
test_data = np.array(
    [
        [1,2],
        [3,1],
        [5,2],
        [3,0]
    ]
)

test_labels = np.array([1,0,0,1])

Manually Calculated Results

* Cohesion: 6.5
* Separation: 4.25
* RMSSTED: 1.275
* RS: 0.395
* DB: 1.228
* XB: 0.382

In [10]:
get_validation_scores(test_data, test_labels)

{'DB': 1.2283204851166758,
 'RMSSTD': 1.2747548783981961,
 'RS': 0.3953488372093023,
 'XB': 0.38235294117647056,
 'calinski_harabaz_score': 1.3076923076923077,
 'cohesion': 6.5,
 'separation': 4.25}

In [11]:
test_data2 = np.array(
    [
        [1,2],
        [3,1],
        [5,2],
        [3,0]
    ]
)

test_labels2 = np.array([1,0,0,2])

Manually Calculated Results

* DB: 0.536
* RS: 0.192

In [12]:
within_cluster_dist_sum_store = {}
get_validation_scores(test_data2, test_labels2)

{'DB': 0.5359848856463295,
 'RMSSTD': 1.118033988749895,
 'RS': 0.7674418604651163,
 'XB': 0.19230769230769232,
 'calinski_harabaz_score': 1.65,
 'cohesion': 2.5,
 'separation': 8.25}

### Calculations for energy profiles

In [13]:
# get the labels for the corresponding subplot
def get_labels(plot_order, labels_dir):
    algo_list = ['kmeans', 'bisectingkmeans', 'gmm']
    current_row = (plot_order-1) // len(algo_list) + 1
    current_col = (plot_order-1) % len(algo_list)
    current_algo = algo_list[current_col]
    current_k = current_row + 1
    return np.load('./%s/%s/params[k=%d].npy' % (labels_dir, current_algo, current_k))

In [14]:
def plot_validation_scores(scores, filename):
    plt.figure(figsize=(16, 16), dpi= 80, facecolor='w', edgecolor='k')

    for idx in range(7):
        score_name = ['cohesion', 'separation', 'calinski_harabaz_score', 'DB', 'RS', 'RMSSTD', 'XB'][idx]
        ax = plt.subplot(3,3,idx+1)

        for i in range(3):
            algo_name = ['K-Means', 'Bisecting K-Means', 'Gaussian Mixture Model'][i]
            values = []
            for j in range(9):
                plot_order = j * 3 + i
                values.append(scores[plot_order][score_name])
            plt.plot(values, label=algo_name)
        plt.title(score_name)
        plt.xticks(range(9), range(2,11))
        plt.xlabel('k')

    plt.savefig('./validation/%s.png' % (filename), bbox_inches='tight')
    plt.close()

In [15]:
settings_list = [
    {
        'profiles': combined_profiles,
        'labels_dir': 'final_labels',
        'save_name': 'Combined Profiles Validation Scores'
    },
    {
        'profiles': residential_profiles,
        'labels_dir': 'residential_labels',
        'save_name': 'Residential Profiles Validation Scores'
    },
    {
        'profiles': non_residential_profiles,
        'labels_dir': 'non_residential_labels',
        'save_name': 'Non-Residential Profiles Validation Scores'
    },
]

In [16]:
for settings in settings_list:

    data = settings['profiles'].iloc[:, 3:3+24].as_matrix()
    scores = []

    start = time.time()
    for i in range(1, 3*9+1):
        labels = get_labels(i, settings['labels_dir'])
        scores.append(get_validation_scores(data, labels))
    print('Time taken for computing scores: %.2f' % (time.time() - start))
    
    plot_validation_scores(scores, settings['save_name'])

Time taken for computing scores: 175.78
Time taken for computing scores: 71.23
Time taken for computing scores: 80.41
